**Intermediate commands**

### DROP

#### Scenario-1

In [ ]:
df.drop('Item_Visibility').display()

#### Scenario-2

In [ ]:
df.drop('Item_Visibility','Item_Type').display()

### DRop_Duplicates

In [ ]:
df.dropDuplicates().display()

#### Scenario - 2

In [ ]:
df.drop_duplicates(subset=['Item_Type']).display()

In [ ]:
df.distinct().display()

### UNION and UNION BY NAME

####Preaparing Dataframes

In [ ]:
data1 = [('1','kad'),
        ('2','sid')]
schema1 = 'id STRING, name STRING'

df1 = spark.createDataFrame(data1,schema1)

data2 = [('3','rahul'),
        ('4','jas')]
schema2 = 'id STRING, name STRING'

df2 = spark.createDataFrame(data2,schema2)



In [ ]:
df1.display()

In [ ]:
df2.display()

### Union

In [ ]:
df1.union(df2).display()

In [ ]:
data1 = [('kad','1',),
        ('sid','2',)]
schema1 = 'name STRING, id STRING'

df1 = spark.createDataFrame(data1,schema1)

df1.display()

In [ ]:
df1.union(df2).display()

### Union by Name

In [ ]:
df1.unionByName(df2).display()

### String Functions

#### Initcap()

In [ ]:
df.select(upper('Item_Type').alias('upper_Item_Type')).display()

### Date Functions

#### Current_Date

In [ ]:
df = df.withColumn('curr_date',current_date())

df.display()

#### Date_Add()

In [ ]:
df = df.withColumn('week_after',date_add('curr_date',7))

df.display()

#### Date_Sub()

In [ ]:
df.withColumn('week_before',date_sub('curr_date',7)).display()

In [ ]:
df = df.withColumn('week_before',date_add('curr_date',-7))

df.display()

### DateDIFF

In [ ]:
df = df.withColumn('datediff',datediff('week_after','curr_date'))

df.display()

### Date_Format()

In [ ]:
df = df.withColumn('week_before',date_format('week_before','dd-MM-yyyy'))

df.display()

### Handling Nulls

#### Dropping NUlls

In [ ]:
df.dropna('all').display()

In [ ]:
df.dropna('any').display()

In [ ]:
df.dropna(subset=['Outlet_Size']).display()

In [ ]:
df.display()

#### Filling Nulls

In [ ]:
df.fillna('NotAvailable').display()

In [ ]:
df.fillna('NotAvailable',subset=['Outlet_Size']).display()

### SPLIT and Indexing

#### SPLIT

In [ ]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')).display()

#### Indexing

In [ ]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')[1]).display()

### Explode

In [ ]:
df_exp = df.withColumn('Outlet_Type',split('Outlet_Type',' '))

df_exp.display()

In [ ]:
df_exp.withColumn('Outlet_Type',explode('Outlet_Type')).display()

In [ ]:
df_exp.display()

In [ ]:
df_exp.withColumn('Type1_flag',array_contains('Outlet_Type','Type1')).display()

### GroupBY

#### Scenario - 1

In [ ]:
df.display()

In [ ]:
df.groupBy('Item_Type').agg(sum('Item_MRP')).display()

#### Scenario - 2

In [ ]:
df.groupBy('Item_Type').agg(avg('Item_MRP')).display()

#### SCenario - 3

In [ ]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP').alias('Total_MRP')).display()

#### Scenario - 4

In [ ]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP'),avg('Item_MRP')).display()

### Collect_List

In [ ]:
data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema = 'user string, book string'

df_book = spark.createDataFrame(data,schema)

df_book.display()

In [ ]:
df_book.groupBy('user').agg(collect_list('book')).display()

In [ ]:
df.select('Item_Type','Outlet_Size','Item_MRP').display()

### PIVOT

In [ ]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).display()

### When-Otherwise

#### Scenario - 1

In [ ]:
df = df.withColumn('veg_flag',when(col('Item_Type')=='Meat','Non-Veg').otherwise('Veg'))

In [ ]:
df.display()

In [ ]:
df.withColumn('veg_exp_flag',when(((col('veg_flag')=='Veg') & (col('Item_MRP')<100)),'Veg_Inexpensive')\
                            .when((col('veg_flag')=='Veg') & (col('Item_MRP')>100),'Veg_Expensive')\
                            .otherwise('Non_Veg')).display()

### JOINS

In [ ]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')]

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING'

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [ ]:
df1.display()

In [ ]:
df2.display()

#### Inner Join

In [ ]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').display()

#### Left Join

In [ ]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'left').display()

#### LEFT JOIN

In [ ]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'right').display()

#### ANTI JOIN

In [ ]:
df1.join(df2,df1['dept_id']==df2['dept_id'],'anti').display()

### WINDOW FUNCTIONS

#### ROW_NUMBER()

In [ ]:
df.display()

In [ ]:
from pyspark.sql.window import Window

In [ ]:
df.withColumn('rowCol',row_number().over(Window.orderBy('Item_Identifier'))).display()

#### RANK VS DENSE RANK

In [ ]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc())))\
        .withColumn('denseRank',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).display()

In [ ]:
df.withColumn('dum',sum('Item_MRP').over(Window.orderBy('Item_Identifier').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()

#### Cumulative Sum

In [ ]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type'))).display()

In [ ]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()

In [ ]:
df.withColumn('totalsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).display()

### USER DEFINED FUNCTIONS (UDF)

#### STEP - 1

In [ ]:
def my_func(x):
    return x*x

#### STEP - 2

In [ ]:
my_udf = udf(my_func)

In [ ]:
df.withColumn('mynewcol',my_udf('Item_MRP')).display()

### DATA WRITING

#### CSV

In [ ]:
df.write.format('csv')\
        .save('/FileStore/tables/CSV/data.csv')

#### APPEND

In [ ]:
df.write.format('csv')\
        .mode('append')\
        .save('/FileStore/tables/CSV/data.csv')

In [ ]:
df.write.format('csv')\
        .mode('append')\
        .option('path','/FileStore/tables/CSV/data.csv')\
        .save()

#### Overwrite

In [ ]:
df.write.format('csv')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### Error

In [ ]:
df.write.format('csv')\
.mode('error')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### Ignore

In [ ]:
df.write.format('csv')\
.mode('ignore')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

#### PARQUET

In [ ]:
df.write.format('parquet')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()